<a href="https://colab.research.google.com/github/kikecastillocolombia/telecomx/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [4]:
import pandas as pd
import requests

url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json'

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)

df.head()


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


#🔧 Transformación

In [9]:
# Mostrar resumen general con info()
df.info()

# Mostrar los tipos de datos de cada columna
print("\nTipos de datos por columna:")
print(df.dtypes)

# Mostrar las primeras 5 filas para ver algunos datos
print("\nPrimeras 5 filas del DataFrame:")
print(df.head())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB

Tipos de datos por columna:
customerID    object
Churn         object
customer      object
phone         object
internet      object
account       object
dtype: object

Primeras 5 filas del DataFrame:
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'M

In [7]:
counts = df['Churn'].value_counts(dropna=False)
percentages = df['Churn'].value_counts(normalize=True, dropna=False) * 100
pd.DataFrame({'counts': counts, 'percentages': percentages})


,counts,percentages
Churn,,
No,5174,71.198569
Yes,1869,25.719004
,224,3.082427


In [8]:
df.loc[~df['Churn'].isin(['Yes', 'No']), 'Churn'].unique()


array([''], dtype=object)

### Comprobación de incoherencias en los datos

In [12]:
#1. Revisar valores nulos por columna
print('Valores nulos por columna:')
print(df.isnull().sum())

#2. Revisar valores vacios('') por columna
print('\nValores vacios('') por columna:')
print((df == '').sum())

# 3. Revisar valores duplicados segun customerID
print("\nNúmero de filas duplicadas según customerID:")
print(df.duplicated(subset=['customerID']).sum())


# 4.Revisar valores únicos en columnas categóricas clave
print('\nValores únicos es "Churn":')
print(df['Churn'].unique())



Valores nulos por columna:
customerID    0
Churn         0
customer      0
phone         0
internet      0
account       0
dtype: int64

Valores vacios() por columna:
customerID      0
Churn         224
customer        0
phone           0
internet        0
account         0
dtype: int64

Número de filas duplicadas según customerID:
0

Valores únicos es "Churn":
['No' 'Yes' '']


In [13]:
import ast

# Función auxiliar para convertir string a dict
def convertir_a_dict(columna):
    return columna.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Convertir strings a diccionarios reales
df['customer'] = convertir_a_dict(df['customer'])
df['phone'] = convertir_a_dict(df['phone'])
df['internet'] = convertir_a_dict(df['internet'])
df['account'] = convertir_a_dict(df['account'])

# Expandir cada columna en múltiples columnas
df_customer = pd.json_normalize(df['customer']).add_prefix('customer_')
df_phone = pd.json_normalize(df['phone']).add_prefix('phone_')
df_internet = pd.json_normalize(df['internet']).add_prefix('internet_')
df_account = pd.json_normalize(df['account']).add_prefix('account_')

# Concatenar columnas expandidas con df original
df = pd.concat([df.drop(['customer', 'phone', 'internet', 'account'], axis=1),
                df_customer, df_phone, df_internet, df_account], axis=1)

# Confirmar estructura final
print("Nuevas columnas del DataFrame:\n")
print(df.columns)


Nuevas columnas del DataFrame:

Index(['customerID', 'Churn', 'customer_gender', 'customer_SeniorCitizen',
       'customer_Partner', 'customer_Dependents', 'customer_tenure',
       'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService',
       'internet_OnlineSecurity', 'internet_OnlineBackup',
       'internet_DeviceProtection', 'internet_TechSupport',
       'internet_StreamingTV', 'internet_StreamingMovies', 'account_Contract',
       'account_PaperlessBilling', 'account_PaymentMethod',
       'account_Charges.Monthly', 'account_Charges.Total'],
      dtype='object')


In [14]:
# 1. Valores nulos
print("Valores nulos por columna:\n")
print(df.isnull().sum())

# 2. Valores vacíos como texto ''
print("\nValores vacíos ('') por columna:\n")
print((df == '').sum())

# 3. Tipos de datos por columna
print("\nTipos de datos por columna:\n")
print(df.dtypes)

# 4. Duplicados (ahora sí debe funcionar)
print("\nNúmero de filas duplicadas:\n")
print(df.duplicated().sum())


Valores nulos por columna:

customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges.Monthly      0
account_Charges.Total        0
dtype: int64

Valores vacíos ('') por columna:

customerID                     0
Churn                        224
customer_gender                0
customer_SeniorCitizen         0
customer_Partner               0
customer_Dependents            0
customer_tenure                0
phone_PhoneService             0
phone_Mul

In [15]:
# 1. Reemplazar valores vacíos en 'Churn' por 'desconocido'
df['Churn'] = df['Churn'].replace('', 'desconocido')

# 2. Pasar todos los valores de texto a minúsculas y quitar espacios
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].str.lower().str.strip()

# 3. Eliminar filas duplicadas
df = df.drop_duplicates()

# 4. Verificar que se aplicaron los cambios
print("Valores únicos en 'Churn':", df['Churn'].unique())
print("\nNúmero de filas después de limpieza:", df.shape[0])


Valores únicos en 'Churn': ['no' 'yes' 'desconocido']

Número de filas después de limpieza: 7267


In [17]:
print(df.columns)


Index(['customerID', 'Churn', 'customer_gender', 'customer_SeniorCitizen',
       'customer_Partner', 'customer_Dependents', 'customer_tenure',
       'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService',
       'internet_OnlineSecurity', 'internet_OnlineBackup',
       'internet_DeviceProtection', 'internet_TechSupport',
       'internet_StreamingTV', 'internet_StreamingMovies', 'account_Contract',
       'account_PaperlessBilling', 'account_PaymentMethod',
       'account_Charges.Monthly', 'account_Charges.Total'],
      dtype='object')


In [19]:
# Convertir la columna de facturación mensual a float y calcular el valor diario
df['Cuentas_Diarias'] = df['account_Charges.Monthly'].astype(float) / 30


In [20]:
df[['account_Charges.Monthly', 'Cuentas_Diarias']].head()


,account_Charges.Monthly,Cuentas_Diarias
0,65.6,2.186667
1,59.9,1.996667
2,73.9,2.463333
3,98.0,3.266667
4,83.9,2.796667


#📊 Carga y análisis

#📄Informe final